In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
base_folder = "/content/drive/MyDrive/-8.0_new"

In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score
import numpy as np

# Specify the base directory containing class folders


# Initialize lists to store DataFrames
dfs = []
labels = []

# Iterate through each folder (each representing a class)
for class_folder in os.listdir(base_folder):
    class_folder_path = os.path.join(base_folder, class_folder)

    # Check if it's a directory
    if os.path.isdir(class_folder_path):
        # Assume each folder has exactly one CSV file
        csv_file = os.path.join(class_folder_path, os.listdir(class_folder_path)[0])

        # Read the CSV file
        try:
            df = pd.read_csv(csv_file)

            # Extract folder name as label
            label = class_folder

            # Add label column to DataFrame
            df['label'] = label

            # Append DataFrame to list
            dfs.append(df)
            labels.append(label)

        except Exception as e:
            print(f"Error reading file '{csv_file}': {e}")

# Concatenate all DataFrames into a single DataFrame
combined_df = pd.concat(dfs, ignore_index=True)

# Check the combined DataFrame
print("Combined DataFrame shape:", combined_df.shape)
feature_cols = ['15', '16', '36', '37', '255', '523', '561', '694', '708', '794']
# Separate features (X) and labels (y)
X = combined_df[feature_cols]
y = combined_df['label']




In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
# Encode labels as categorical (required for LSTM)
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
num_classes = len(label_encoder.classes_)

# Normalize data (assuming features are numerical)
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Reshape data for LSTM input: (samples, time steps, features)
# Assuming 10 time steps (adjust based on your data characteristics)
time_steps = 30  # Adjust as per your data
num_features = X.shape[1]

def reshape_data_for_lstm(X, y, time_steps):
    X_lstm = []
    y_lstm = []
    for i in range(len(X) - time_steps + 1):
        X_lstm.append(X[i:i + time_steps])
        y_lstm.append(y[i + time_steps - 1])
    return np.array(X_lstm), np.array(y_lstm)

# Reshape data for LSTM input
X_lstm, y_lstm = reshape_data_for_lstm(X, y, time_steps)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_lstm, y_lstm, test_size=0.2, random_state=42)

# Define LSTM model
model = Sequential([
    LSTM(64, input_shape=(time_steps, num_features)),
    Dense(num_classes, activation='softmax')  # Output layer with softmax activation
])

# Compile the model

model.compile(optimizer='Adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Print model summary
model.summary()

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, min_delta=0.001)

# Train the model with early stopping
train_history=model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.3,
          verbose=1, callbacks=[early_stopping])

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {accuracy:.2f}")

# Optional: Predict using the trained model
# y_pred = model.predict(X_test)
# Convert predictions back to labels if needed
# y_pred_labels = label_encoder.inverse_transform(np.argmax(y_pred, axis=1))

In [ ]:
import matplotlib.pyplot as plt

train_loss = train_history.history['loss']
train_accuracy = train_history.history['accuracy']
val_loss = train_history.history['val_loss']
val_accuracy = train_history.history['val_accuracy']
# Plot training and validation loss
plt.plot(train_loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title("SNR 0")
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Evaluate the model
y_pred = model.predict(X_test)
y_pred_class = np.argmax(y_pred, axis=1)

# Accuracy
accuracy = accuracy_score(y_test, y_pred_class)
print(f"Test Accuracy: {accuracy:.2f}")

# Classification Report
print("Classification Report:")
print(classification_report(y_test, y_pred_class))

# Confusion Matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_class))

# Precision, Recall, F1-score
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(y_test, y_pred_class, average='weighted')
recall = recall_score(y_test, y_pred_class, average='weighted')
f1 = f1_score(y_test, y_pred_class, average='weighted')

print(f"Test Precision: {precision:.2f}")
print(f"Test Recall: {recall:.2f}")
print(f"Test F1-score: {f1:.2f}")